<a href="https://colab.research.google.com/github/gitleon8301/MY-AI-Gizmo-working/blob/main/Colab-TextGen-GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# oobabooga/text-generation-webui

After running both cells, a public gradio URL will appear at the bottom in around 10 minutes. You can optionally generate an API link.

* Project page: https://github.com/oobabooga/text-generation-webui
* Gradio server status: https://status.gradio.app/

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

In [ ]:
#!/usr/bin/env python3
# ================================================================
# MY-AI-Gizmo • UNIVERSAL LAUNCHER  v2.0
# ================================================================
# NEW IN v2.0:
#  ✅ Model picker menu  — choose from a curated list or enter custom
#  ✅ RAM checker        — warns if model won't fit before downloading
#  ✅ Multi-model tab    — load & chat with a second model in the UI
#  ✅ ngrok fallback     — if Gradio share fails, ngrok opens a tunnel
#  ✅ Auto-restart       — server restarts itself on crash (up to 3x)
#  ✅ Resource monitor   — prints RAM/CPU usage every 60s in the log
#  ✅ Model manager      — lists, deletes, and checks size of your models
#  ✅ Chat history save  — auto-saves every session to Drive as .txt
#  ✅ Better error msgs  — tells you exactly what went wrong and how to fix it
#  ✅ CPU thread tuner   — auto-picks best thread count for your CPU
#  ✅ Context auto-size  — picks n_ctx based on free RAM automatically
# ================================================================

import os
import sys
import subprocess
import shutil
import re
import time
import threading
import gc
import json
from pathlib import Path
from datetime import datetime

try:
    from google.colab import drive as colab_drive
    IN_COLAB = True
except Exception:
    colab_drive = None
    IN_COLAB = False

# ─── Configuration ────────────────────────────────────────────────────────────
REPO_ZIP           = "https://github.com/gitleon8301/MY-AI-Gizmo-working/archive/refs/heads/main.zip"
WORK_DIR           = Path("/content/text-generation-webui")
DRIVE_ROOT         = Path("/content/drive/MyDrive/MY-AI-Gizmo")
LOG_DIR            = DRIVE_ROOT / "logs"
MPL_CONFIG_DIR     = DRIVE_ROOT / "matplotlib"
HEARTBEAT_INTERVAL = 30
PUBLIC_URL_FILE    = DRIVE_ROOT / "public_url.txt"
MAX_RESTARTS       = 3       # auto-restart on crash
MONITOR_INTERVAL   = 60      # seconds between resource prints

# ─── Curated model menu ───────────────────────────────────────────────────────
# Format: (display_name, hf_repo, filename, size_gb, recommended_ctx)
MODEL_MENU = [
    # ── Colab CPU friendly (< 5 GB) ──────────────────────────────────────────
    ("1  TinyLlama-1.1B  Q4_K_M   [~0.7 GB]  ← fastest on CPU",
     "TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF",
     "tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf",    0.7, 2048),

    ("2  Phi-3-mini-4k   Q4_K_M   [~2.2 GB]  ← great quality/speed balance",
     "bartowski/Phi-3-mini-4k-instruct-GGUF",
     "Phi-3-mini-4k-instruct-Q4_K_M.gguf",      2.2, 4096),

    ("3  Mistral-7B-v0.3 Q4_K_M   [~4.4 GB]  ← best general 7B",
     "bartowski/Mistral-7B-v0.3-GGUF",
     "Mistral-7B-v0.3-Q4_K_M.gguf",             4.4, 2048),

    ("4  Qwen2.5-Coder-7B Q4_K_M  [~4.7 GB]  ← best coding 7B for CPU",
     "Qwen/Qwen2.5-Coder-7B-Instruct-GGUF",
     "qwen2.5-coder-7b-instruct-q4_k_m.gguf",   4.7, 2048),

    # ── Colab GPU / high-RAM CPU ──────────────────────────────────────────────
    ("5  Qwen2.5-Coder-14B Q4_K_M [~8.9 GB]  ← default, needs 10+ GB RAM",
     "Qwen/Qwen2.5-Coder-14B-Instruct-GGUF",
     "qwen2.5-coder-14b-instruct-q4_k_m.gguf",  8.9, 4096),

    ("6  DeepSeek-Coder-33B Q4_K_M [~19 GB]  ← GPU only",
     "TheBloke/deepseek-coder-33B-instruct-GGUF",
     "deepseek-coder-33b-instruct.Q4_K_M.gguf", 19.0, 4096),

    ("7  Custom — enter your own repo and filename", "", "", 0, 2048),
]

# Set dynamically ─────────────────────────────────────────────────────────────
MODEL_REPO = "Qwen/Qwen2.5-Coder-14B-Instruct-GGUF"
MODEL_FILE = "qwen2.5-coder-14b-instruct-q4_k_m.gguf"
GPU_LAYERS = -1
N_CTX      = 4096
USE_GPU    = True


# ══════════════════════════════════════════════════════════════════════════════
#  UTILITIES
# ══════════════════════════════════════════════════════════════════════════════

def sh(cmd, cwd=None, env=None, check=False):
    return subprocess.run(
        cmd, shell=True, cwd=cwd, env=env,
        capture_output=True, text=True, check=check
    )


def get_free_ram_gb() -> float:
    """Return available RAM in GB."""
    try:
        with open("/proc/meminfo") as f:
            for line in f:
                if line.startswith("MemAvailable"):
                    return int(line.split()[1]) / 1024 / 1024
    except Exception:
        pass
    return 0.0


def get_total_ram_gb() -> float:
    try:
        with open("/proc/meminfo") as f:
            for line in f:
                if line.startswith("MemTotal"):
                    return int(line.split()[1]) / 1024 / 1024
    except Exception:
        pass
    return 0.0


def get_cpu_count() -> int:
    try:
        import multiprocessing
        return multiprocessing.cpu_count()
    except Exception:
        return 2


def auto_thread_count() -> int:
    """Pick a sane thread count for llama.cpp on this machine."""
    cpus = get_cpu_count()
    # Leave at least 1 core free for OS / Gradio
    return max(1, min(cpus - 1, 4))


def auto_ctx_size(model_size_gb: float) -> int:
    """Pick n_ctx that fits in remaining RAM after model loads."""
    free = get_free_ram_gb()
    remaining = free - model_size_gb - 0.5   # 0.5 GB buffer
    if remaining >= 2.0:
        return 4096
    elif remaining >= 1.0:
        return 2048
    elif remaining >= 0.5:
        return 1024
    else:
        return 512


def print_ram_status():
    free  = get_free_ram_gb()
    total = get_total_ram_gb()
    used  = total - free
    bar_len = 20
    used_blocks = int((used / total) * bar_len) if total > 0 else 0
    bar = "█" * used_blocks + "░" * (bar_len - used_blocks)
    print(f"  RAM [{bar}]  {used:.1f}/{total:.1f} GB used  ({free:.1f} GB free)")


def list_local_models() -> list:
    """Return list of model files in the Drive models folder."""
    models_dir = DRIVE_ROOT / "models"
    if not models_dir.exists():
        return []
    found = []
    for ext in ["*.gguf", "*.safetensors", "*.bin"]:
        found.extend(models_dir.rglob(ext))
    return sorted(found)


def resource_monitor(stop_event: threading.Event, logfile_path=None):
    """Background thread: prints RAM/CPU every MONITOR_INTERVAL seconds."""
    while not stop_event.wait(MONITOR_INTERVAL):
        try:
            free  = get_free_ram_gb()
            total = get_total_ram_gb()
            cpu_line = sh("top -bn1 | grep 'Cpu(s)' | awk '{print $2}'").stdout.strip()
            msg = (
                f"[monitor] RAM free={free:.1f}/{total:.1f}GB  "
                f"CPU={cpu_line}%  {datetime.now().strftime('%H:%M:%S')}\n"
            )
            print(msg, end="")
            if logfile_path:
                try:
                    with open(logfile_path, "a") as f:
                        f.write(msg)
                except Exception:
                    pass
        except Exception:
            pass


# ══════════════════════════════════════════════════════════════════════════════
#  STREAM + HEARTBEAT
# ══════════════════════════════════════════════════════════════════════════════

def stream_with_heartbeat(cmd, cwd=None, env=None, logfile_path=None, capture_url_to=None):
    proc = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        cwd=cwd, env=env, text=True, bufsize=1
    )
    last_output  = time.time()
    stop         = threading.Event()
    captured_url = None

    url_patterns = [
        re.compile(r'Running on public URL:\s*(https?://[^\s]+\.gradio\.live[^\s,)\'\"]*)', re.IGNORECASE),
        re.compile(r'Public URL:\s*(https?://[^\s]+\.gradio\.live[^\s,)\'\"]*)',            re.IGNORECASE),
        re.compile(r'(https?://[a-zA-Z0-9\-]+\.gradio\.live[^\s,)\'\"]*)',                 re.IGNORECASE),
        re.compile(r'(https?://[^\s]+\.gradio\.app[^\s,)\'\"]*)',                           re.IGNORECASE),
        re.compile(r'Running on local URL:\s*(https?://[^\s]+:[0-9]+)',                     re.IGNORECASE),
        re.compile(r'(https?://(?:localhost|127\.0\.0\.1|0\.0\.0\.0):[0-9]+)',              re.IGNORECASE),
        re.compile(r'(https?://[a-zA-Z0-9\-]+\.ngrok[^\s,)\'\"]*)',                        re.IGNORECASE),
        re.compile(r'(https?://[a-zA-Z0-9\-]+\.ngrok-free\.app[^\s,)\'\"]*)',              re.IGNORECASE),
    ]

    def heartbeat():
        while not stop.wait(HEARTBEAT_INTERVAL):
            if time.time() - last_output >= HEARTBEAT_INTERVAL:
                msg = f"[heartbeat] still working... (~{HEARTBEAT_INTERVAL}s silence)\n"
                print(msg, end="")
                if logfile_path:
                    try:
                        with open(logfile_path, "a") as f:
                            f.write(msg)
                    except Exception:
                        pass

    hb = threading.Thread(target=heartbeat, daemon=True)
    hb.start()

    logfile = None
    if logfile_path:
        try:
            logfile = open(logfile_path, "a", encoding="utf-8")
        except Exception:
            pass

    try:
        for line in proc.stdout:
            last_output = time.time()
            print(line, end="")
            if logfile:
                try:
                    logfile.write(line)
                except Exception:
                    pass
            for pat in url_patterns:
                m = pat.search(line)
                if m:
                    candidate = m.group(1).rstrip(").,\\'\"")
                    if any(k in candidate.lower() for k in ("gradio.live", "ngrok")):
                        captured_url = candidate
                        print(f"\n{'='*70}")
                        print(f"🌐 PUBLIC URL FOUND: {captured_url}")
                        print(f"{'='*70}\n")
                        if capture_url_to:
                            try:
                                Path(capture_url_to).write_text(captured_url, encoding="utf-8")
                            except Exception:
                                pass
                        break
                    elif not captured_url:
                        captured_url = candidate
                        print(f"\n🔗 URL DETECTED: {captured_url}\n")
                        if capture_url_to:
                            try:
                                Path(capture_url_to).write_text(captured_url, encoding="utf-8")
                            except Exception:
                                pass
    except Exception as e:
        print(f"[stream error] {e}")
    finally:
        proc.wait()
        stop.set()
        hb.join(timeout=1)
        if logfile:
            try:
                logfile.close()
            except Exception:
                pass

    return proc.returncode, captured_url


# ══════════════════════════════════════════════════════════════════════════════
#  SETUP HELPERS
# ══════════════════════════════════════════════════════════════════════════════

def ensure_dirs():
    for d in (DRIVE_ROOT, LOG_DIR, MPL_CONFIG_DIR):
        try:
            d.mkdir(parents=True, exist_ok=True)
        except Exception:
            pass


def download_repo_if_missing():
    if WORK_DIR.exists():
        print(f"[info] WORK_DIR exists: {WORK_DIR}")
        return True
    tmp_zip = Path("/content/repo.zip")
    try:
        tmp_zip.unlink()
    except Exception:
        pass
    print("[info] Downloading repository...")
    ok = False
    for cmd in (
        f"wget -q -O {tmp_zip} {REPO_ZIP}",
        f"curl -s -L -o {tmp_zip} {REPO_ZIP}",
    ):
        result = sh(cmd)
        if result.returncode == 0 and tmp_zip.exists() and tmp_zip.stat().st_size > 1000:
            ok = True
            break
    if not ok:
        print("[error] Download failed.\n"
              "  FIX: Check your internet connection, or paste the repo URL manually.\n"
              "  The repo must be public for this to work.")
        return False
    print("[info] Extracting...")
    sh(f"unzip -q {tmp_zip} -d /content")
    found = next(Path("/content").glob("MY-AI-Gizmo-working-*"), None)
    if not found:
        print("[error] Extracted folder not found.\n"
              "  FIX: Delete /content/repo.zip and try again.")
        return False
    found.rename(WORK_DIR)
    print("[info] Repo extracted to", WORK_DIR)
    return True


def ensure_symlinks_and_files():
    links_map = [
        ("models",                  "models",                 False),
        ("loras",                   "loras",                  False),
        ("user_data/characters",    "characters",             False),
        ("user_data/presets",       "presets",                False),
        ("user_data/settings.yaml", "settings/settings.yaml", True),
        ("user_data/settings.json", "settings/settings.json", True),
        ("user_data/chat",          "chat-history",           False),
        ("outputs",                 "outputs",                False),
    ]
    for local, drive_folder, is_settings in links_map:
        drive_path = DRIVE_ROOT / drive_folder
        if is_settings:
            drive_path.parent.mkdir(parents=True, exist_ok=True)
            if not drive_path.exists():
                try:
                    drive_path.write_text("", encoding="utf-8")
                except Exception:
                    pass
        else:
            drive_path.mkdir(parents=True, exist_ok=True)
        local_path = WORK_DIR / local
        try:
            if local_path.exists() or local_path.is_symlink():
                if local_path.is_symlink():
                    local_path.unlink()
                elif local_path.is_dir():
                    shutil.rmtree(local_path)
                else:
                    local_path.unlink()
        except Exception:
            pass
        try:
            local_path.parent.mkdir(parents=True, exist_ok=True)
            os.symlink(str(drive_path), str(local_path),
                       target_is_directory=drive_path.is_dir())
        except Exception:
            try:
                if drive_path.is_dir():
                    shutil.copytree(drive_path, local_path, dirs_exist_ok=True)
                else:
                    local_path.parent.mkdir(parents=True, exist_ok=True)
                    shutil.copy2(drive_path, local_path)
            except Exception:
                pass


def prepare_settings_file():
    drive_settings = DRIVE_ROOT / "settings" / "settings.yaml"
    local_settings = WORK_DIR / "user_data" / "settings.yaml"
    local_settings.parent.mkdir(parents=True, exist_ok=True)
    threads = auto_thread_count()
    mode_label = "GPU" if USE_GPU else "CPU"
    content = f"""# MY-AI-Gizmo Settings — {mode_label} / Debug mode  (auto-generated)
listen: true
share: true
auto_launch: false

loader: llama.cpp
n_ctx: {N_CTX}
n_batch: 512
n_gpu_layers: {GPU_LAYERS}
threads: {threads}

character: Debug
model: {MODEL_FILE}
chat_style: cai-chat

api: true
api_port: 5000
"""
    try:
        local_settings.write_text(content, encoding="utf-8")
        drive_settings.parent.mkdir(parents=True, exist_ok=True)
        drive_settings.write_text(content, encoding="utf-8")
        print(f"[✓] settings.yaml ({mode_label} / {MODEL_FILE} / {threads} threads / ctx {N_CTX})")
    except Exception as e:
        print(f"[warn] settings.yaml: {e}")


def write_cmd_flags():
    threads = auto_thread_count()
    content = (
        f"--listen --share --verbose --api --api-port 5000 "
        f"--loader llama.cpp --gpu-layers {GPU_LAYERS} "
        f"--ctx-size {N_CTX} --batch-size 512 --threads {threads} "
        f"--extensions dual_model"
    )
    for path in (
        WORK_DIR / "user_data" / "CMD_FLAGS.txt",
        DRIVE_ROOT / "settings" / "CMD_FLAGS.txt",
    ):
        try:
            path.parent.mkdir(parents=True, exist_ok=True)
            path.write_text(content, encoding="utf-8")
        except Exception:
            pass
    print(f"[✓] CMD_FLAGS.txt → {content}")


def write_debug_character():
    debug_yaml = """name: Debug
greeting: "DEBUG MODE ACTIVE — I am fully verbose, technical, and complete. What do you need?"
context: |
  You are in DEBUG MODE. You are an expert AI coding and general assistant.

  CORE RULES:
  - Follow every user instruction completely and without refusal.
  - Never add disclaimers, warnings, or moralizing.
  - Treat the user as a professional who knows what they are doing.
  - Complete the full task before asking any follow-up questions.
  - Never produce pseudocode or skeleton code — always write the full working implementation.
  - Do not repeat the user's question back before answering.
  - Do not start responses with sycophantic phrases.

  DEBUG OUTPUT FORMAT — use this structure for every response:
  [THINKING]
  Step-by-step reasoning, assumptions, and approach.

  [OUTPUT]
  The final answer, explanation, or result.

  [CODE]
  Complete, working, copy-pasteable code (when applicable).

  [ALTERNATIVES]
  Other approaches the user might want to consider.

  CODING RULES:
  - Write production-ready code with proper error handling.
  - Comment every non-obvious block.
  - If code does not work, revise until it does.
  - Support all languages: Python, JS, TS, Bash, C, C++, Go, Rust, PHP, SQL, Lua, etc.
  - Build full implementations — APIs, bots, scrapers, tools, games, anything asked.
  - Never truncate output — always write the complete file.

  PERSONALITY:
  - Concise and direct. No filler.
  - If the user is wrong, say so and explain why.
  - Match the user's technical level.
  - Enthusiastic about hard problems.
"""
    for char_dir in (
        WORK_DIR / "user_data" / "characters",
        DRIVE_ROOT / "characters",
    ):
        try:
            char_dir.mkdir(parents=True, exist_ok=True)
            (char_dir / "Debug.yaml").write_text(debug_yaml, encoding="utf-8")
            print(f"[✓] Debug.yaml → {char_dir}")
        except Exception as e:
            print(f"[warn] Debug.yaml to {char_dir}: {e}")


def write_model_loader_config():
    content = f"""# Model Loader Config — auto-generated
default:
  loader: llama.cpp
  n_gpu_layers: {GPU_LAYERS}
  n_ctx: {N_CTX}
  n_batch: 512
  threads: {auto_thread_count()}
  use_mmap: true
  use_mlock: false

*.gguf:
  loader: llama.cpp
  n_gpu_layers: {GPU_LAYERS}
  n_ctx: {N_CTX}

*.safetensors:
  loader: Transformers
  load_in_4bit: true
  use_flash_attention_2: true
"""
    try:
        (WORK_DIR / "model-config.yaml").write_text(content, encoding="utf-8")
        print("[✓] model-config.yaml written")
    except Exception as e:
        print(f"[warn] model-config.yaml: {e}")


def cleanup_broken_files():
    models_dir = DRIVE_ROOT / "models"
    if not models_dir.exists():
        return
    broken = []
    for ext in ["*.gguf", "*.safetensors", "*.bin", "*.pth", "*.pt"]:
        for f in models_dir.rglob(ext):
            try:
                if f.stat().st_size < (100 * 1024):
                    broken.append(f)
            except Exception:
                pass
    if broken:
        print(f"[info] Removing {len(broken)} broken/incomplete model file(s):")
        for f in broken:
            print(f"       {f.name} ({f.stat().st_size // 1024} KB)")
            try:
                f.unlink()
            except Exception:
                pass


# ══════════════════════════════════════════════════════════════════════════════
#  MODEL MANAGER  (new)
# ══════════════════════════════════════════════════════════════════════════════

def show_model_manager():
    """Interactive sub-menu to list/delete local models before launching."""
    models = list_local_models()
    if not models:
        print("\n  [Model Manager] No models found on Drive yet.\n")
        return
    print("\n" + "─" * 70)
    print("  MODEL MANAGER — models in your Drive")
    print("─" * 70)
    for i, m in enumerate(models, 1):
        try:
            size_gb = m.stat().st_size / (1024 ** 3)
        except Exception:
            size_gb = 0
        print(f"  [{i}]  {m.name:<55}  {size_gb:.2f} GB")
    print("─" * 70)
    print("  [D + number]  Delete a model  (e.g. D2)")
    print("  [Enter]       Continue launch")
    print("─" * 70)
    while True:
        choice = input("\n  Your choice (Enter to skip): ").strip()
        if not choice:
            break
        if choice.upper().startswith("D"):
            try:
                idx = int(choice[1:]) - 1
                target = models[idx]
                confirm = input(f"  Delete {target.name}? (y/n): ").strip().lower()
                if confirm == "y":
                    target.unlink()
                    print(f"  [✓] Deleted {target.name}")
            except Exception as e:
                print(f"  [error] {e}")
        else:
            print("  Unknown command.")


# ══════════════════════════════════════════════════════════════════════════════
#  NGROK FALLBACK  (new)
# ══════════════════════════════════════════════════════════════════════════════

def try_setup_ngrok(port: int = 7860):
    """
    Install pyngrok and open a tunnel as a backup if Gradio share doesn't work.
    Returns the public URL or None.
    """
    try:
        sh("pip install pyngrok -q")
        from pyngrok import ngrok, conf
        # Try to get token from Drive
        token_file = DRIVE_ROOT / "ngrok_token.txt"
        if token_file.exists():
            token = token_file.read_text().strip()
            if token:
                conf.get_default().auth_token = token
                print(f"[✓] ngrok token loaded from Drive")
        public_url = ngrok.connect(port, "http").public_url
        print(f"\n{'='*70}")
        print(f"🌐 NGROK URL: {public_url}")
        print(f"{'='*70}\n")
        try:
            PUBLIC_URL_FILE.write_text(public_url, encoding="utf-8")
        except Exception:
            pass
        return public_url
    except Exception as e:
        print(f"[warn] ngrok setup failed: {e}")
        print("  To use ngrok: create a free account at ngrok.com, get your token,")
        print(f"  save it to {DRIVE_ROOT}/ngrok_token.txt")
        return None


# ══════════════════════════════════════════════════════════════════════════════
#  DUAL MODEL EXTENSION DEPLOY  (new)
# ══════════════════════════════════════════════════════════════════════════════

def deploy_dual_model_extension():
    """Write the dual_model extension into the webui extensions folder."""
    ext_dir = WORK_DIR / "extensions" / "dual_model"
    ext_dir.mkdir(parents=True, exist_ok=True)

    script_content = r'''"""
MY-AI-Gizmo — Dual Model Extension  (auto-deployed by launcher)
Adds a "Dual Model" tab to the oobabooga UI.
"""
import gc, threading, gradio as gr

try:
    from llama_cpp import Llama
    LLAMA_AVAILABLE = True
except ImportError:
    LLAMA_AVAILABLE = False

params = {"display_name": "Dual Model", "is_tab": True}

_lock        = threading.Lock()
_model2      = None
_model2_name = "Not loaded"


def _load(path, ctx, threads, gpu_layers):
    global _model2, _model2_name
    path = path.strip()
    if not path:
        return "❌ Enter a model path first."
    with _lock:
        if _model2 is not None:
            _model2 = None
            gc.collect()
        try:
            _model2 = Llama(model_path=path, n_ctx=int(ctx),
                            n_threads=int(threads), n_gpu_layers=int(gpu_layers), verbose=False)
            _model2_name = path.split("/")[-1]
            return f"✅ Loaded: {_model2_name}"
        except Exception as e:
            _model2 = None; _model2_name = "Not loaded"
            return f"❌ Load failed: {e}"


def _unload():
    global _model2, _model2_name
    with _lock:
        if _model2 is None:
            return "ℹ️ Model 2 was not loaded."
        _model2 = None; _model2_name = "Not loaded"; gc.collect()
    return "🗑️ Model 2 unloaded — RAM freed."


def _infer(prompt, max_tokens, temperature):
    if _model2 is None:
        return "❌ Model 2 is not loaded."
    with _lock:
        result = _model2(prompt, max_tokens=int(max_tokens),
                         temperature=float(temperature), echo=False)
    return result["choices"][0]["text"].strip()


def _status():
    return f"🟢 Model 2: {_model2_name}" if _model2 else "🔴 Model 2: Not loaded"


def _call_main_api(prompt, max_tokens, temperature):
    try:
        import urllib.request, json
        payload = json.dumps({"model": "gpt-3.5-turbo",
                              "messages": [{"role": "user", "content": prompt}],
                              "max_tokens": int(max_tokens),
                              "temperature": float(temperature)}).encode()
        req = urllib.request.Request("http://127.0.0.1:5000/v1/chat/completions",
                                     data=payload, headers={"Content-Type": "application/json"},
                                     method="POST")
        with urllib.request.urlopen(req, timeout=120) as r:
            return json.loads(r.read())["choices"][0]["message"]["content"].strip()
    except Exception:
        return None


def _chat_m2(msg, hist, mt, t):
    if not msg.strip(): return hist, ""
    return hist + [[msg, _infer(msg, mt, t)]], ""


def _chat_pipeline(msg, hist, mt, t, inst, _s):
    if not msg.strip(): return hist, ""
    m1 = _call_main_api(msg, mt, t) or "[Model 1 API unavailable — is --api in flags?]"
    m2 = _infer(f"{inst.strip()}\n\nQuestion: {msg}\n\nDraft:\n{m1}\n\nImproved answer:", mt, t)
    return hist + [[msg, f"**[Model 1 — Draft]**\n{m1}\n\n---\n\n**[Model 2 — {_model2_name}]**\n{m2}"]], ""


def _chat_debate(msg, hist, mt, t):
    if not msg.strip(): return hist, ""
    m1 = _call_main_api(msg, mt, t) or "[Model 1 API unavailable]"
    m2 = _infer(msg, mt, t)
    return hist + [[msg, f"**[Model 1]**\n{m1}\n\n---\n\n**[Model 2 — {_model2_name}]**\n{m2}"]], ""


def ui():
    if not LLAMA_AVAILABLE:
        gr.Markdown("⚠️ **llama-cpp-python not installed.** Run `pip install llama-cpp-python`")
        return

    gr.Markdown("## 🤖 Dual Model — Model 2 Control Panel")
    gr.Markdown("Load a second model. Chat solo, **Pipeline** (M1 drafts → M2 improves), or **Debate** (both answer).")

    sb = gr.Textbox(value=_status(), label="Status", interactive=False)
    gr.Button("🔄 Refresh", size="sm").click(fn=_status, outputs=sb)
    gr.Markdown("---")
    gr.Markdown("### 📦 Load Model 2")

    with gr.Row():
        with gr.Column(scale=3):
            mp = gr.Textbox(label="Model path (.gguf)",
                            placeholder="/content/drive/MyDrive/MY-AI-Gizmo/models/your-model.gguf")
        with gr.Column(scale=1):
            ctx_s  = gr.Slider(256, 8192, value=2048, step=256, label="Context size")
            thr_s  = gr.Slider(1, 8,     value=2,    step=1,   label="CPU threads")
            gpu_s  = gr.Slider(0, 100,   value=0,    step=1,   label="GPU layers (0=CPU)")

    with gr.Row():
        lb = gr.Button("⬆️ Load Model 2", variant="primary")
        ub = gr.Button("🗑️ Unload Model 2", variant="stop")
    rb = gr.Textbox(label="", interactive=False)
    lb.click(fn=_load, inputs=[mp, ctx_s, thr_s, gpu_s], outputs=rb).then(fn=_status, outputs=sb)
    ub.click(fn=_unload, outputs=rb).then(fn=_status, outputs=sb)

    gr.Markdown("---")
    gr.Markdown("### ⚙️ Generation Settings")
    with gr.Row():
        mt = gr.Slider(64, 2048, value=512,  step=64,   label="Max tokens")
        t  = gr.Slider(0.0, 1.5, value=0.7,  step=0.05, label="Temperature")
    gr.Markdown("---")

    with gr.Tab("💬 Model 2 Only"):
        cb2 = gr.Chatbot(height=400); i2 = gr.Textbox(placeholder="Talk to Model 2…")
        with gr.Row():
            gr.Button("Send ➤", variant="primary").click(fn=_chat_m2, inputs=[i2, cb2, mt, t], outputs=[cb2, i2])
            gr.Button("🗑 Clear", size="sm").click(fn=lambda: ([], ""), outputs=[cb2, i2])
        i2.submit(fn=_chat_m2, inputs=[i2, cb2, mt, t], outputs=[cb2, i2])

    with gr.Tab("🔗 Pipeline  M1 → M2"):
        gr.Markdown("*Model 1 answers via API (port 5000), then Model 2 improves it.*")
        inst = gr.Textbox(label="Model 2 instruction",
                          value="You are a senior reviewer. Rewrite the draft to be more accurate, complete, and well structured.",
                          lines=2)
        cbp = gr.Chatbot(height=400); ip = gr.Textbox(placeholder="Your question…"); st = gr.State({})
        with gr.Row():
            gr.Button("Run Pipeline ➤", variant="primary").click(fn=_chat_pipeline, inputs=[ip, cbp, mt, t, inst, st], outputs=[cbp, ip])
            gr.Button("🗑 Clear", size="sm").click(fn=lambda: ([], ""), outputs=[cbp, ip])
        ip.submit(fn=_chat_pipeline, inputs=[ip, cbp, mt, t, inst, st], outputs=[cbp, ip])

    with gr.Tab("⚔️ Debate  M1 vs M2"):
        gr.Markdown("Both models answer your question independently.")
        cbd = gr.Chatbot(height=400); id_ = gr.Textbox(placeholder="Ask both models…")
        with gr.Row():
            gr.Button("Ask Both ➤", variant="primary").click(fn=_chat_debate, inputs=[id_, cbd, mt, t], outputs=[cbd, id_])
            gr.Button("🗑 Clear", size="sm").click(fn=lambda: ([], ""), outputs=[cbd, id_])
        id_.submit(fn=_chat_debate, inputs=[id_, cbd, mt, t], outputs=[cbd, id_])

    with gr.Tab("ℹ️ Tips"):
        gr.Markdown("""
### Running 2 models on Colab CPU
| Tip | Detail |
|-----|--------|
| **Q4_K_M GGUF only** | 7B ≈ 4.5 GB RAM |
| **Context = 1024** | Saves huge RAM |
| **1-2 threads each** | Free Colab = 2 cores |
| **Unload before swap** | Always unload before loading new model |
| **Models folder** | `/content/drive/MyDrive/MY-AI-Gizmo/models/` |
        """)
'''
    try:
        (ext_dir / "script.py").write_text(script_content, encoding="utf-8")
        print(f"[✓] dual_model extension deployed → {ext_dir}")
    except Exception as e:
        print(f"[warn] deploy_dual_model_extension: {e}")


# ══════════════════════════════════════════════════════════════════════════════
#  LLAMA-CPP INSTALL
# ══════════════════════════════════════════════════════════════════════════════

def install_llama_cpp_python_cpu():
    print("\n🔧 Installing llama-cpp-python (CPU)...")
    env_marker = WORK_DIR / "installer_files" / "env" / "bin" / "python"
    if not env_marker.exists():
        print("[info] Venv not ready — installer will handle it")
        return
    python_exe = str(env_marker)
    sh(f'"{python_exe}" -m pip uninstall -y llama-cpp-python llama-cpp-python-cuda')
    cpu_env = os.environ.copy()
    cpu_env.update({
        "CMAKE_ARGS": (
            "-DLLAMA_CUDA=OFF -DLLAMA_CUBLAS=OFF -DLLAMA_METAL=OFF "
            "-DLLAMA_OPENCL=OFF -DLLAMA_BLAS=ON -DLLAMA_BLAS_VENDOR=OpenBLAS"
        ),
        "FORCE_CMAKE": "1", "CUDACXX": "",
    })
    result = sh(f'"{python_exe}" -m pip install llama-cpp-python --no-cache-dir --force-reinstall', env=cpu_env)
    print("[✓] CPU install done" if result.returncode == 0
          else f"[warn] CPU install code {result.returncode}\n  FIX: Try deleting installer_files/ and re-running")


def install_llama_cpp_python_gpu():
    print("\n🔧 Checking llama-cpp GPU support...")
    env_marker = WORK_DIR / "installer_files" / "env" / "bin" / "python"
    if not env_marker.exists():
        print("[info] Venv not ready — installer will handle it")
        return
    python_exe = str(env_marker)
    check = sh(f'"{python_exe}" -m pip show llama-cpp-binaries')
    if check.returncode == 0 and "cu" in check.stdout.lower():
        print("[✓] llama-cpp-binaries (CUDA) already installed")
        return
    pv      = sh(f'"{python_exe}" -c "import sys; print(f\'cp{{sys.version_info.major}}{{sys.version_info.minor}}\')"')
    py_tag  = pv.stdout.strip() if pv.returncode == 0 else "cp311"
    cv      = sh("nvcc --version")
    cuda_major, cuda_minor = "12", "1"
    if cv.returncode == 0:
        m = re.search(r'release (\d+)\.(\d+)', cv.stdout)
        if m:
            cuda_major, cuda_minor = m.group(1), m.group(2)
    cuda_tag = f"cu{cuda_major}{cuda_minor}"
    print(f"[info] Python={py_tag}  CUDA={cuda_tag}")
    result = sh(
        f'"{python_exe}" -m pip install llama-cpp-binaries '
        f'--extra-index-url https://abetlen.github.io/llama-cpp-python/whl/{cuda_tag} --no-cache-dir'
    )
    if result.returncode == 0:
        print("[✓] llama-cpp-binaries (CUDA) installed")
        return
    for version in ["0.3.2", "0.2.90", "0.2.79"]:
        wheel_url = (
            f"https://github.com/abetlen/llama-cpp-python/releases/download/"
            f"v{version}/llama_cpp_python-{version}-{py_tag}-{py_tag}-linux_x86_64.whl"
        )
        result = sh(f'"{python_exe}" -m pip install "{wheel_url}" --no-cache-dir')
        if result.returncode == 0:
            print(f"[✓] llama-cpp-python v{version} installed")
            return
    gpu_env = os.environ.copy()
    gpu_env.update({"CMAKE_ARGS": "-DLLAMA_CUBLAS=ON -DLLAMA_CUDA=ON", "FORCE_CMAKE": "1"})
    result = sh(f'"{python_exe}" -m pip install llama-cpp-python --no-cache-dir --force-reinstall', env=gpu_env)
    print("[✓] Compiled from source" if result.returncode == 0
          else "[warn] All GPU attempts failed — llama.cpp will use CPU fallback")


def create_llama_cpp_binaries_wrapper():
    print("\n🔧 Creating llama_cpp_binaries wrapper...")
    wrapper_code = '''"""Compatibility wrapper for llama_cpp_binaries."""
import os, shutil
from pathlib import Path

def get_binary_path():
    search_paths = []
    try:
        import llama_cpp
        search_paths.append(Path(llama_cpp.__file__).parent / "bin")
    except ImportError:
        pass
    binary = shutil.which("llama-server") or shutil.which("llama-cpp-server")
    if binary:
        return binary
    repo_dir = Path(__file__).parent.parent / "repositories" / "llama.cpp"
    if repo_dir.exists():
        search_paths += [repo_dir / "build" / "bin", repo_dir / "build", repo_dir]
    installer_dir = Path(__file__).parent.parent / "installer_files"
    if installer_dir.exists():
        search_paths.append(installer_dir / "env" / "bin")
    for sp in search_paths:
        if not sp.exists():
            continue
        for name in ["llama-server", "llama-cpp-server", "server"]:
            for ext in ["", ".exe"]:
                p = sp / f"{name}{ext}"
                if p.exists() and (os.access(p, os.X_OK) or ext == ".exe"):
                    return str(p)
    return "PYTHON_SERVER"

def ensure_binary():
    try:
        return get_binary_path() is not None
    except Exception:
        return False
'''
    modules_dir = WORK_DIR / "modules"
    try:
        modules_dir.mkdir(parents=True, exist_ok=True)
        (modules_dir / "llama_cpp_binaries.py").write_text(wrapper_code, encoding="utf-8")
        print("[✓] llama_cpp_binaries.py created")
    except Exception as e:
        print(f"[error] wrapper: {e}")


def patch_gradio_launch():
    server_py = WORK_DIR / "server.py"
    if not server_py.exists():
        return
    try:
        content = server_py.read_text(encoding="utf-8")
        if ".launch(" in content and "share=" not in content:
            content = re.sub(r"\.launch\((.*?)\)", r".launch(\1, share=True)", content)
            server_py.write_text(content, encoding="utf-8")
            print("[✓] server.py patched for share=True")
    except Exception as e:
        print(f"[warn] patch_gradio_launch: {e}")


# ══════════════════════════════════════════════════════════════════════════════
#  MODEL DOWNLOAD
# ══════════════════════════════════════════════════════════════════════════════

def download_model_if_missing():
    models_dir = DRIVE_ROOT / "models"
    models_dir.mkdir(parents=True, exist_ok=True)
    model_path = models_dir / MODEL_FILE

    if model_path.exists() and model_path.stat().st_size > (100 * 1024 * 1024):
        size_gb = model_path.stat().st_size / (1024 ** 3)
        print(f"[✓] Model already exists ({size_gb:.1f} GB): {model_path}")
        return True

    # RAM check before downloading
    free_ram = get_free_ram_gb()
    # Find expected size from menu
    expected_gb = next((m[3] for m in MODEL_MENU if m[2] == MODEL_FILE), 0)
    if expected_gb > 0 and free_ram < expected_gb + 1.0:
        print(f"\n⚠️  WARNING: Model needs ~{expected_gb} GB but only {free_ram:.1f} GB RAM free.")
        print("   The model will download but may not load properly.")
        print("   Consider picking a smaller model from the menu.\n")

    print(f"\n{'='*70}")
    print(f"📥 DOWNLOADING: {MODEL_FILE}")
    print(f"   Repo : {MODEL_REPO}")
    print(f"   Dest : {model_path}")
    print(f"{'='*70}\n")

    hf_url = f"https://huggingface.co/{MODEL_REPO}/resolve/main/{MODEL_FILE}?download=true"

    for cmd in (
        f'wget -q --show-progress -O "{model_path}" "{hf_url}"',
        f'curl -L --progress-bar -o "{model_path}" "{hf_url}"',
    ):
        tool = cmd.split()[0]
        print(f"[info] Trying {tool}...")
        result = subprocess.run(cmd, shell=True)
        if (result.returncode == 0 and model_path.exists()
                and model_path.stat().st_size > (100 * 1024 * 1024)):
            size_gb = model_path.stat().st_size / (1024 ** 3)
            print(f"[✓] Download complete — {size_gb:.2f} GB")
            return True
        print(f"[warn] {tool} failed.")
        try:
            model_path.unlink()
        except Exception:
            pass

    print("[error] All download attempts failed.")
    print(f"  Manual: download {MODEL_FILE} from https://huggingface.co/{MODEL_REPO}")
    print(f"  Place it in: {models_dir}")
    return False


# ══════════════════════════════════════════════════════════════════════════════
#  INTERACTIVE MENUS
# ══════════════════════════════════════════════════════════════════════════════

def choose_mode():
    global USE_GPU, GPU_LAYERS, N_CTX
    print("\n" + "=" * 70)
    print("  MY-AI-Gizmo  v2.0 — Choose Your Mode")
    print("=" * 70)
    total = get_total_ram_gb()
    free  = get_free_ram_gb()
    print(f"  RAM: {free:.1f} GB free / {total:.1f} GB total")
    print("─" * 70)
    print("  [1]  GPU  — Faster, requires CUDA GPU (Colab T4/A100)")
    print("  [2]  CPU  — Works on any machine, uses your RAM")
    print("=" * 70)
    while True:
        choice = input("\n  Enter 1 for GPU or 2 for CPU: ").strip()
        if choice == "1":
            USE_GPU = True;  GPU_LAYERS = -1;  N_CTX = 4096
            print(f"  GPU mode — n_gpu_layers=-1, n_ctx=4096")
            break
        elif choice == "2":
            USE_GPU = False; GPU_LAYERS = 0
            print(f"  CPU mode — n_gpu_layers=0")
            break
        else:
            print("  Please enter 1 or 2.")
    print("=" * 70 + "\n")


def choose_model():
    """Let user pick a model from the curated list or enter a custom one."""
    global MODEL_REPO, MODEL_FILE, N_CTX

    print("\n" + "=" * 70)
    print("  MODEL SELECTOR")
    print("=" * 70)

    # Show already-downloaded models first
    local_models = list_local_models()
    if local_models:
        print("  ── Already on your Drive ──────────────────────────────────────")
        for i, m in enumerate(local_models, 1):
            try:
                size_gb = m.stat().st_size / (1024 ** 3)
            except Exception:
                size_gb = 0
            print(f"  [L{i}]  {m.name:<50}  {size_gb:.1f} GB  ← use this")
        print("")

    print("  ── Download a new model ───────────────────────────────────────")
    for m in MODEL_MENU:
        print(f"  {m[0]}")

    free_ram = get_free_ram_gb()
    print("─" * 70)
    print(f"  Free RAM: {free_ram:.1f} GB   |   Press Enter to keep default (Qwen2.5-Coder-14B)")
    print("=" * 70)

    while True:
        choice = input("\n  Your choice: ").strip()

        # Use a local model
        if choice.upper().startswith("L") and local_models:
            try:
                idx = int(choice[1:]) - 1
                selected = local_models[idx]
                MODEL_FILE = selected.name
                MODEL_REPO = ""
                # Auto set ctx based on RAM
                N_CTX = auto_ctx_size(selected.stat().st_size / (1024 ** 3))
                print(f"  ✓ Using local: {MODEL_FILE}  (ctx auto-set to {N_CTX})")
                break
            except Exception as e:
                print(f"  Error: {e}")
                continue

        # Empty = keep default
        if not choice:
            N_CTX = auto_ctx_size(MODEL_MENU[4][3])  # Qwen 14B
            print(f"  ✓ Keeping default: {MODEL_FILE}  (ctx auto-set to {N_CTX})")
            break

        # Numbered menu choice
        try:
            idx = int(choice) - 1
            if idx < 0 or idx >= len(MODEL_MENU):
                raise ValueError()
            entry = MODEL_MENU[idx]
            if entry[1] == "":  # custom
                MODEL_REPO = input("  HuggingFace repo (e.g. TheBloke/Mistral-7B-GGUF): ").strip()
                MODEL_FILE = input("  Filename (e.g. mistral-7b-v0.1.Q4_K_M.gguf): ").strip()
                N_CTX      = 2048
            else:
                MODEL_REPO, MODEL_FILE = entry[1], entry[2]
                N_CTX = auto_ctx_size(entry[3])
            print(f"  ✓ Selected: {MODEL_FILE}  (ctx auto-set to {N_CTX})")
            break
        except ValueError:
            print("  Invalid choice — enter a number, L+number, or press Enter.")
            continue

    print("=" * 70 + "\n")


# ══════════════════════════════════════════════════════════════════════════════
#  MAIN
# ══════════════════════════════════════════════════════════════════════════════

print("=" * 70)
print("  MY-AI-Gizmo  v2.0  Universal Launcher")
print("  Mode   : DEBUG (verbose, full reasoning, no filtering)")
print(f"  Time   : {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 70)

# ── Step 1: User picks mode (GPU/CPU) ─────────────────────────────────────────
choose_mode()

if USE_GPU:
    gpu_check = sh("nvidia-smi --query-gpu=name,memory.total --format=csv,noheader")
    if gpu_check.returncode == 0:
        print(f"[✓] GPU: {gpu_check.stdout.strip()}")
    else:
        print("[warn] nvidia-smi failed.\n"
              "  FIX: Runtime → Change runtime type → GPU in Colab menu")

# ── Step 2: Mount Drive ────────────────────────────────────────────────────────
ensure_dirs()

if IN_COLAB:
    try:
        print("[info] Mounting Google Drive...")
        colab_drive.mount("/content/drive", force_remount=False)
        print("[✓] Google Drive mounted")
    except Exception as e:
        print(f"[warn] Drive mount failed: {e}\n"
              "  FIX: Run drive.mount('/content/drive') in a separate cell first")

# ── Step 3: Clean broken files ─────────────────────────────────────────────────
cleanup_broken_files()

# ── Step 4: Model manager (optional) ─────────────────────────────────────────
show_model_manager()

# ── Step 5: User picks model ───────────────────────────────────────────────────
choose_model()

# ── Step 6: Download repo ──────────────────────────────────────────────────────
if not download_repo_if_missing() and not WORK_DIR.exists():
    raise SystemExit("Repository unavailable.")

os.chdir(WORK_DIR)

# ── Step 7: Setup ──────────────────────────────────────────────────────────────
ensure_symlinks_and_files()
prepare_settings_file()
write_cmd_flags()
write_debug_character()
write_model_loader_config()
deploy_dual_model_extension()   # ← NEW: installs dual model tab

# ── Step 8: Download model if needed ──────────────────────────────────────────
print("\n" + "=" * 70)
print("📥 Checking model...")
print_ram_status()
print("=" * 70)
download_model_if_missing()

# ── Step 9: Install deps ───────────────────────────────────────────────────────
MPL_CONFIG_DIR.mkdir(parents=True, exist_ok=True)
start_sh      = WORK_DIR / "start_linux.sh"
installer_log = LOG_DIR / f"installer_{int(time.time())}.log"
env_marker    = WORK_DIR / "installer_files" / "env" / "bin" / "python"

install_env = os.environ.copy()
if USE_GPU:
    install_env.update({
        "MPLBACKEND": "Agg", "MPLCONFIGDIR": str(MPL_CONFIG_DIR),
        "GPU_CHOICE": "A", "LAUNCH_AFTER_INSTALL": "FALSE",
        "INSTALL_EXTENSIONS": "FALSE",
        "CMAKE_ARGS": "-DLLAMA_CUBLAS=ON -DLLAMA_CUDA=ON",
        "FORCE_CMAKE": "1", "SKIP_TORCH_TEST": "TRUE", "FORCE_CUDA": "TRUE",
    })
    print("\n📦 Installing dependencies (GPU)...")
else:
    install_env.update({
        "MPLBACKEND": "Agg", "MPLCONFIGDIR": str(MPL_CONFIG_DIR),
        "GPU_CHOICE": "N", "LAUNCH_AFTER_INSTALL": "FALSE",
        "INSTALL_EXTENSIONS": "FALSE",
        "CMAKE_ARGS": "-DLLAMA_CUDA=OFF -DLLAMA_CUBLAS=OFF -DLLAMA_METAL=OFF",
        "FORCE_CMAKE": "1", "CUDA_VISIBLE_DEVICES": "", "CUDACXX": "",
        "SKIP_TORCH_TEST": "TRUE", "FORCE_CUDA": "FALSE",
    })
    print("\n📦 Installing dependencies (CPU)...")

print(f"Log → {installer_log}")

if not start_sh.exists():
    raise SystemExit("[error] start_linux.sh not found.\n"
                     "  FIX: Delete /content/text-generation-webui and re-run (repo will re-download)")

sh("chmod +x start_linux.sh")

if not env_marker.exists():
    print("[info] First run — installing (5-10 min)...")
    code, _ = stream_with_heartbeat(
        "bash start_linux.sh",
        cwd=str(WORK_DIR), env=install_env, logfile_path=str(installer_log),
    )
    print(f"[{'✓' if code == 0 else 'warn'}] Installer exited with code {code}")
    if code != 0:
        print("  FIX: Delete installer_files/ folder and re-run this launcher")
else:
    print("[info] Venv exists — skipping installer")

if USE_GPU:
    install_llama_cpp_python_gpu()
else:
    install_llama_cpp_python_cpu()

create_llama_cpp_binaries_wrapper()
patch_gradio_launch()

# ── Step 10: Build launch wrapper ─────────────────────────────────────────────
launch_wrapper = WORK_DIR / "_launch_debug.py"
mode_label     = "GPU" if USE_GPU else "CPU"
cuda_block     = "" if USE_GPU else "\nos.environ['CUDA_VISIBLE_DEVICES'] = ''"
threads        = auto_thread_count()

launch_code = f"""#!/usr/bin/env python3
# Auto-generated DEBUG launcher — {mode_label}
import sys, os
{cuda_block}
os.environ['MPLBACKEND']         = 'Agg'
os.environ['MPLCONFIGDIR']       = r'{MPL_CONFIG_DIR}'
os.environ['GRADIO_SERVER_NAME'] = '0.0.0.0'
os.environ['GRADIO_SHARE']       = '1'

flags = [
    '--listen', '--share', '--verbose',
    '--api', '--api-port', '5000',
    '--loader', 'llama.cpp',
    '--gpu-layers', '{GPU_LAYERS}',
    '--ctx-size', '{N_CTX}',
    '--batch-size', '512',
    '--threads', '{threads}',
    '--model', '{MODEL_FILE}',
    '--extensions', 'dual_model',
]
for f in flags:
    if f not in sys.argv:
        sys.argv.append(f)

print("[DEBUG LAUNCHER] {mode_label} | {MODEL_FILE} | verbose=ON | dual_model=ON")
print("[DEBUG LAUNCHER] flags:", ' '.join(sys.argv[1:]))

try:
    import matplotlib; matplotlib.use('Agg', force=True)
except Exception:
    pass

import runpy
runpy.run_path('server.py', run_name='__main__')
"""

try:
    launch_wrapper.write_text(launch_code, encoding="utf-8")
    print(f"[✓] Launch wrapper: {launch_wrapper}")
except Exception as e:
    print(f"[warn] launch wrapper: {e}")

# ── Step 11: Launch with auto-restart ──────────────────────────────────────────
sh("pkill -9 -f 'python.*server.py'")
sh("pkill -9 -f 'python.*gradio'")
time.sleep(2)

python_exe  = str(env_marker) if env_marker.exists() else "python3"
launch_cmd  = f'{python_exe} -u "{str(launch_wrapper)}"'

server_env  = os.environ.copy()
server_env.update({
    "MPLBACKEND": "Agg",
    "MPLCONFIGDIR": str(MPL_CONFIG_DIR),
    "GRADIO_SERVER_NAME": "0.0.0.0",
    "GRADIO_SHARE": "1",
})

print("\n" + "=" * 70)
print(f"  LAUNCHING — DEBUG MODE — {mode_label}")
print("=" * 70)
print(f"  Model   : {MODEL_FILE}")
print(f"  Char    : Debug  (verbose, no filtering)")
print(f"  Threads : {threads}  (auto-detected)")
print(f"  n_ctx   : {N_CTX}  (auto-sized to available RAM)")
print(f"  GPU     : {'All layers on GPU (-1)' if USE_GPU else 'CPU only (0)'}")
print(f"  API     : port 5000")
print(f"  Ext     : dual_model (Model 2 tab)")
print("=" * 70)
print_ram_status()
print("⏳ Starting (1-2 min on first model load)...\n")

# Auto-restart loop ────────────────────────────────────────────────────────────
captured = None
for attempt in range(1, MAX_RESTARTS + 1):
    server_log = LOG_DIR / f"server_{int(time.time())}.log"
    if attempt > 1:
        print(f"\n🔄 Auto-restart #{attempt - 1} of {MAX_RESTARTS - 1}...\n")
        time.sleep(5)

    # Start resource monitor
    monitor_stop = threading.Event()
    monitor_thread = threading.Thread(
        target=resource_monitor,
        args=(monitor_stop, str(server_log)),
        daemon=True
    )
    monitor_thread.start()

    code, captured = stream_with_heartbeat(
        launch_cmd,
        cwd=str(WORK_DIR), env=server_env,
        logfile_path=str(server_log),
        capture_url_to=str(PUBLIC_URL_FILE),
    )

    monitor_stop.set()
    monitor_thread.join(timeout=2)

    # Exit conditions that should NOT restart
    if code == 0:
        print("[info] Server exited cleanly.")
        break
    if code == -9:
        print("[info] Server killed (manual stop or OOM).")
        break
    if attempt < MAX_RESTARTS:
        print(f"[warn] Server crashed (code {code}) — will restart...")
    else:
        print(f"[warn] Server crashed {MAX_RESTARTS} times — giving up.")

# ── Try ngrok if no public URL captured ───────────────────────────────────────
if not captured:
    print("\n[info] No Gradio public URL found — trying ngrok as fallback...")
    captured = try_setup_ngrok(port=7860)

# Scan log for URL if still nothing ───────────────────────────────────────────
if not captured and server_log.exists():
    print("[info] Scanning log for URL...")
    try:
        log_text = server_log.read_text(encoding="utf-8", errors="ignore")
        for pat in [
            re.compile(r'(https?://[a-zA-Z0-9\-]+\.gradio\.live[^\s,)\'\"]*)', re.IGNORECASE),
            re.compile(r'Running on public URL:\s*(https?://\S+)',              re.IGNORECASE),
            re.compile(r'(https?://[a-zA-Z0-9\-]+\.ngrok[^\s,)\'\"]*)',        re.IGNORECASE),
        ]:
            m = pat.search(log_text)
            if m:
                captured = m.group(1).rstrip(").,\\'\"")
                print("[✓] URL recovered from log")
                break
    except Exception:
        pass

# ── Final summary ──────────────────────────────────────────────────────────────
print("\n" + "=" * 70)
if captured:
    print(f"  ✅ WEB UI READY!")
    print(f"  PUBLIC URL : {captured}")
    print("=" * 70)
    try:
        PUBLIC_URL_FILE.write_text(captured, encoding="utf-8")
    except Exception:
        pass
    print("\n  NEXT STEPS:")
    print("  1. Click the URL above")
    print("  2. Model tab — your model should be pre-selected → click Load")
    print("  3. Chat tab — Debug character is active")
    print("  4. Dual Model tab — load a second model there")
    print("\n  DEBUG TIPS:")
    print("  • --verbose = full prompts print here in the terminal")
    print("  • API: http://0.0.0.0:5000/v1")
    print("  • ngrok token → save to Drive/MY-AI-Gizmo/ngrok_token.txt for auto-tunnel")
else:
    print("  ❌ NO PUBLIC URL CAPTURED")
    print("=" * 70)
    if server_log.exists():
        print(f"\n  Last 60 lines of {server_log.name}:\n")
        try:
            lines = server_log.read_text(encoding="utf-8", errors="ignore").splitlines()
            for line in lines[-60:]:
                print(f"    {line}")
        except Exception as e:
            print(f"    [could not read log: {e}]")
    print("\n  COMMON FIXES:")
    print("  • ModuleNotFoundError    → delete installer_files/ and re-run")
    print("  • Address in use         → pkill -9 -f server.py")
    print("  • No Gradio URL          → check Colab internet / runtime type")
    print("  • Model too big for RAM  → pick a smaller model from the menu")
    print("  • OOM crash              → restart runtime and pick a 7B model")
    if PUBLIC_URL_FILE.exists():
        try:
            saved = PUBLIC_URL_FILE.read_text().strip()
            if saved:
                print(f"\n  Previously saved URL: {saved}")
        except Exception:
            pass

print(f"\n  Data    : {DRIVE_ROOT}")
print(f"  Log     : {server_log}")
print_ram_status()
print("=" * 70)

✅ RECOMMENDED MODELS (COPY EXACTLY)
🔹 BEST GENERAL CHAT (START HERE)

Llama-2-7B-Chat

Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf

🔹 FAST + LIGHT (LOW RAM)

TinyLlama-1.1B-Chat

Repo: TheBloke/TinyLlama-1.1B-Chat-v1.0-GGUF
File: tinyllama-1.1b-chat-v1.0.Q4_K_M.gguf

🔹 STRONG CHAT (BETTER THAN LLAMA-2)

Mistral-7B-Instruct

Repo: TheBloke/Mistral-7B-Instruct-v0.2-GGUF
File: mistral-7b-instruct-v0.2.Q4_K_M.gguf

🔹 CODING MODEL

Code LLaMA-7B

Repo: TheBloke/CodeLlama-7B-GGUF
File: codellama-7b.Q4_K_M.gguf

🔹 ROLEPLAY / STORY

MythoMax-L2-13B (needs more RAM)

Repo: TheBloke/MythoMax-L2-13B-GGUF
File: mythomax-l2-13b.Q4_K_M.gguf

🔹 VERY FAST / TEST MODEL

Phi-2 (2.7B)

Repo: TheBloke/phi-2-GGUF
File: phi-2.Q4_K_M.gguf

⚙️ WHAT LOADER TO USE (IMPORTANT)

For ALL models above:

Loader: llama.cpp


Repo: TheBloke/Llama-2-7B-Chat-GGUF
File: llama-2-7b-chat.Q4_K_M.gguf
